# Prediction of bike rental counts using a regression model

### Import Libraries

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso 
from sklearn.svm import SVR

from sklearn import model_selection
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import TimeSeriesSplit
import numpy as np 
from numpy import sqrt

### Load Dataset

In [2]:
df = pd.read_csv('/Users/tanchaud/CAB_Berlin/Module_1/Project_3/Bike-Sharing-Dataset/hour.csv')

In [3]:
df = df.rename(columns={'dteday':'Date','season':'Season' ,'yr':'Year', 'mnth': 'Month', 
            'hr':'Hour','holiday':'Holiday', 'weekday':'Day_of_the_Week','workingday': 'Working_Day',
            'weathersit':'Weather_Situation','temp':'Temperature',
            'atemp':'Feels_like Temperature','hum':'Humidity','windspeed': 'Wind_Speed',
            'casual':'Casual_Users','registered':'Registered_Users','cnt':'Total_Users'})


In [ ]:
df['Total_Users'].plot()

In [6]:
df.dtypes

Season                    category
Year                         int64
Month                        int64
Hour                         int64
Holiday                      int64
Day_of_the_Week              int64
Working_Day                  int64
Weather_Situation         category
Feels_like Temperature     float64
Humidity                   float64
Wind_Speed                 float64
Total_Users                  int64
dtype: object

In [7]:
# Feature Selection
X = df[['Season', 'Weather_Situation', 'Feels_like Temperature', 'Humidity', 'Wind_Speed']]
X.head(3)

,Season,Weather_Situation,Feels_like Temperature,Humidity,Wind_Speed
Date,,,,,
2011-01-01,1,1,0.2879,0.81,0.0
2011-01-01,1,1,0.2727,0.80,0.0
2011-01-01,1,1,0.2727,0.80,0.0


In [ ]:
## Encoding of categorical variables 

# define one hot encoding
encoder = OneHotEncoder(sparse=False)
# transform data
onehot = encoder.fit_transform(X)


In [9]:
X.head(3)

,Season,Weather_Situation,Feels_like Temperature,Humidity,Wind_Speed
Date,,,,,
2011-01-01,1,1,0.2879,0.81,0.0
2011-01-01,1,1,0.2727,0.80,0.0
2011-01-01,1,1,0.2727,0.80,0.0


### Train Test Split

In [ ]:
X_test.head()

In [ ]:
%matplotlib inline




In [ ]:
models[2][1]

In [ ]:
regressor.coef_[6]

In [ ]:
df.head()